In [1]:
from neo4j import GraphDatabase
import pandas as pd
from tabulate import tabulate
import os
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [2]:
directory = os.getcwd()
path = directory + '/Data/'

## Connecting to database in neo4j

In [3]:
driver=GraphDatabase.driver("bolt://localhost:7687",auth=("neo4j",""))
session = driver.session(database="graphwooptions")

## Louvain 

In [4]:
query='''CALL gds.louvain.write({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  includeIntermediateCommunities: false,
  seedProperty: '',
  writeProperty: 'louvain'
});'''
with driver.session(database="graphwooptions") as session:
    session.run(query)

## Label propagation algorithm

In [5]:
query='''CALL gds.labelPropagation.write({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'REVERSE',
      properties: {
        value: {
          property: 'value',
          defaultValue: 1
        }
      }
    }
  },
  relationshipWeightProperty: 'value',
  seedProperty: '',
  writeProperty: 'lpa'
});'''
with driver.session(database="graphwooptions") as session:
    session.run(query)

## Strongly connected components

In [6]:
query='''CALL gds.alpha.scc.write({
  nodeProjection: 'company',
  relationshipProjection: {
    relType: {
      type: 'Holds',
      orientation: 'REVERSE',
      properties: {}
    }
  },
  writeProperty: 'scc'
});'''
with driver.session(database="graphwooptions") as session:
    session.run(query)

## Analysing scc, the biggest community of scc has 129 nodes and has scc=14082

In [7]:
query='''MATCH (n)-[rel:Holds]->(m) where n.scc=14082 RETURN distinct n.id as id, n.company as company, n.scc as scc'''
    
    
with driver.session(database="graphwooptions") as session:
    scc=pd.DataFrame([dict(record) for record in session.run(query)])
    scc=scc[['scc','company','id']]

In [8]:
scc.to_csv(path+'scc.csv')